In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Set up parameters

In [2]:
trainDataDir = "/Users/SamZhang/Documents/Capstone/dataset/small/train"

In [3]:
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .2, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", trainDataDir + "/spam/SMS_train.spam", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", trainDataDir + "/ham/SMS_train.ham", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 256, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 20, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [4]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.2
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.1
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/small/train/ham/SMS_train.ham
NUM_CHECKPOINTS=5
NUM_EPOCHS=20
NUM_FILTERS=256
POSITIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/small/train/spam/SMS_train.spam



# Process data

In [5]:
def processData():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
    print(x_text[0])

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    
    return x_train, y_train, x_dev, y_dev, vocab_processor

In [6]:
x_train, y_train, x_dev, y_dev, vocab_processor = processData()

Loading data...
free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question \( std txt rate \) t c 's apply 08452810075over18 's
Vocabulary Size: 7814
Train/Dev split: 3568/892


# Building CNN

In [7]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnnmodel"))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2018-03-15T17:48:03.683413: step 97, loss 0.330531, acc 0.96875
2018-03-15T17:48:04.629976: step 98, loss 0.362217, acc 0.953125
2018-03-15T17:48:05.577202: step 99, loss 0.322987, acc 0.9375
2018-03-15T17:48:06.519368: step 100, loss 0.257919, acc 0.96875

Evaluation:
2018-03-15T17:48:08.847431: step 100, loss 0.308056, acc 0.969731

Saved model checkpoint to /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/model-100

2018-03-15T17:48:09.944949: step 101, loss 0.279902, acc 0.953125
2018-03-15T17:48:10.895282: step 102, loss 0.384854, acc 0.96875
2018-03-15T17:48:11.843925: step 103, loss 0.233529, acc 0.96875
2018-03-15T17:48:12.801791: step 104, loss 0.232146, acc 0.96875
2018-03-15T17:48:14.009841: step 105, loss 0.443532, acc 0.960938
2018-03-15T17:48:15.067472: step 106, loss 0.36805, acc 0.945312
2018-03-15T17:48:16.284688: step 107, loss 0.234471, acc 0.96875
2018-03-15T17:48:17.549798: step 108, loss 0.285291, acc 0.976562
2018-03-15T17:48:18.811334: step 10

2018-03-15T17:50:08.341658: step 217, loss 0.191203, acc 0.984375
2018-03-15T17:50:09.309278: step 218, loss 0.167548, acc 0.992188
2018-03-15T17:50:10.268578: step 219, loss 0.160504, acc 1
2018-03-15T17:50:11.210428: step 220, loss 0.17248, acc 0.992188
2018-03-15T17:50:12.151756: step 221, loss 0.203864, acc 0.96875
2018-03-15T17:50:13.102264: step 222, loss 0.15715, acc 1
2018-03-15T17:50:14.056976: step 223, loss 0.20358, acc 0.976562
2018-03-15T17:50:14.888307: step 224, loss 0.189547, acc 0.982143
2018-03-15T17:50:15.871752: step 225, loss 0.185083, acc 0.984375
2018-03-15T17:50:16.845629: step 226, loss 0.19873, acc 0.984375
2018-03-15T17:50:17.805668: step 227, loss 0.190343, acc 0.992188
2018-03-15T17:50:18.774465: step 228, loss 0.176944, acc 0.992188
2018-03-15T17:50:19.760785: step 229, loss 0.169672, acc 0.992188
2018-03-15T17:50:20.786222: step 230, loss 0.179228, acc 0.992188
2018-03-15T17:50:21.914691: step 231, loss 0.1802, acc 0.984375
2018-03-15T17:50:22.889826: ste

2018-03-15T17:52:16.250851: step 343, loss 0.133276, acc 1
2018-03-15T17:52:17.300527: step 344, loss 0.128649, acc 1
2018-03-15T17:52:18.428066: step 345, loss 0.12953, acc 1
2018-03-15T17:52:19.503304: step 346, loss 0.128673, acc 1
2018-03-15T17:52:20.576532: step 347, loss 0.125813, acc 1
2018-03-15T17:52:21.577730: step 348, loss 0.151708, acc 0.984375
2018-03-15T17:52:22.585581: step 349, loss 0.129339, acc 1
2018-03-15T17:52:23.646870: step 350, loss 0.136685, acc 0.992188
2018-03-15T17:52:24.639136: step 351, loss 0.141615, acc 0.992188
2018-03-15T17:52:25.677102: step 352, loss 0.146848, acc 0.992188
2018-03-15T17:52:26.685365: step 353, loss 0.131489, acc 1
2018-03-15T17:52:27.675339: step 354, loss 0.13246, acc 0.992188
2018-03-15T17:52:28.663471: step 355, loss 0.125443, acc 1
2018-03-15T17:52:29.656338: step 356, loss 0.122022, acc 1
2018-03-15T17:52:30.653042: step 357, loss 0.14591, acc 0.992188
2018-03-15T17:52:31.631935: step 358, loss 0.134307, acc 1
2018-03-15T17:52:

2018-03-15T17:54:34.430885: step 473, loss 0.104869, acc 0.992188
2018-03-15T17:54:35.473833: step 474, loss 0.0985968, acc 1
2018-03-15T17:54:36.639961: step 475, loss 0.101305, acc 1
2018-03-15T17:54:37.590911: step 476, loss 0.0943907, acc 1
2018-03-15T17:54:38.651629: step 477, loss 0.0941423, acc 1
2018-03-15T17:54:39.712960: step 478, loss 0.109951, acc 0.992188
2018-03-15T17:54:40.919685: step 479, loss 0.0934749, acc 1
2018-03-15T17:54:42.098462: step 480, loss 0.10352, acc 1
2018-03-15T17:54:43.160217: step 481, loss 0.0962551, acc 1
2018-03-15T17:54:44.201648: step 482, loss 0.0968699, acc 1
2018-03-15T17:54:45.244851: step 483, loss 0.0985117, acc 1
2018-03-15T17:54:46.524977: step 484, loss 0.0947701, acc 1
2018-03-15T17:54:47.643117: step 485, loss 0.0988312, acc 0.992188
2018-03-15T17:54:48.797995: step 486, loss 0.0944142, acc 1
2018-03-15T17:54:49.833843: step 487, loss 0.0940382, acc 1
2018-03-15T17:54:50.854279: step 488, loss 0.091814, acc 1
2018-03-15T17:54:51.99574

# Testing model

In [8]:
from sklearn import metrics
import csv

In [9]:
testDataDir = "/Users/SamZhang/Documents/Capstone/dataset/small/test"

In [10]:
tf.flags.DEFINE_string("positive_test_data_file", testDataDir + "/spam/SMS_test.spam", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_test_data_file", testDataDir + "/ham/SMS_test.ham", "Data source for the negative data.")

# Eval Parameters
tf.flags.DEFINE_string("checkpoint_dir", "/Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/", "Checkpoint directory from training run")

In [11]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_DIR=/Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.2
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.1
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/small/train/ham/SMS_train.ham
NEGATIVE_TEST_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/small/test/ham/SMS_test.ham
NUM_CHECKPOINTS=5
NUM_EPOCHS=20
NUM_FILTERS=256
POSITIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/small/train/spam/SMS_train.spam
POSITIVE_TEST_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/small/test/spam/SMS_test.spam



In [17]:
x_raw, y_test = data_helpers.load_data_and_labels(FLAGS.positive_test_data_file, FLAGS.negative_test_data_file)
y_test = np.argmax(y_test, axis=1) #ham = 0, spam = 1
print(x_raw[0], y_test[0])

six chances to win cash ! from 100 to 20 , 000 pounds txt csh11 and send to 87575 cost 150p day , 6days , 16 tsandcs apply reply hl 4 info 1


In [13]:
vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

In [14]:
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)

In [15]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

INFO:tensorflow:Restoring parameters from /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/model-500
Total number of test examples: 1114
Accuracy: 0.964991


In [16]:
# Save the evaluation to a csv
title = np.column_stack(('text', 'prediction', 'label'))
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions, y_test))
out_path = os.path.join(FLAGS.checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))

with open(out_path, 'w') as f:
    csv.writer(f).writerows(title)
    csv.writer(f).writerows(predictions_human_readable)

Saving evaluation to /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/../prediction.csv
